In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]

In [ ]:
billboard_csv = 'billboard_hot_100_1991-01-05_to_2022-10-01.csv'
df = pd.read_csv(billboard_csv, parse_dates=['date'])

View data before any processing:

In [ ]:
df

Reorder columns:

In [ ]:
df = df.filter(['date', 'pos', 'pos_prev', 'pos_peak', 'weeks', 'artist', 'song'])

In [ ]:
df.head()

Add some columns for convenience:
* Add a "year", derived from the "date" column, and use it as the index.
* Add an "artist_song" column, derived from the "artist" and "song" columns. (We'll use this to determine unique songs in each year.)

In [ ]:
df['year'] = df['date'].dt.year
df.set_index('year', inplace=True)
df['artist_song'] = df['artist'] + ': ' + df['song']
df[['artist', 'song', 'artist_song']].head()

Remove the original 'artist' and 'song' columns, since we now have the combined 'artist_song', and remove all rows with year 2022, since we don't have complete data for this year.

In [ ]:
df = df.drop(columns=['artist', 'song']).drop(2022)
df

Find the number of unique charting songs (by 'artist'/'song') for each year:

In [ ]:
_ = df.groupby(['year'])['artist_song'].nunique().plot.bar(title='Number of unique charting songs by year')

Sort all chart weeks by year (earliest first), then by peak chart position (lowest first), then by the number of consecutive weeks in the charts (most weeks first).

Finally, drop all rows (chart weeks) with duplicate song, keeping only the first (highest & longest-charting week of the earliest charting year) of each duplicate.

In [ ]:
num_rows_raw = len(df)
df = df.sort_values(['year', 'pos_peak', 'weeks'], ascending=[True, True, False]).drop_duplicates(['artist_song'])
num_rows_unique_by_year = len(df)
df

Let's look at a single year to make sure the low peak-position & high week counts are at the top:

In [ ]:
df[df.index == 2021]

Check how many rows we dropped, and make sure each song is unique across the whole dataframe:

In [ ]:
print('Num rows (raw):', num_rows_raw)
print('Num rows (with unique songs per-year):', num_rows_unique_by_year)
assert df['artist_song'].count() == df['artist_song'].nunique()

One thing that sticks out to me here is that more recent years have more unique charting songs, by a substantial margin.
My naive interpretation is a combination of our tastes are becoming more eclectic and independent artists getting more exposure through Soundcloud/Bandcamp. But maybe it has more to do with the music industry/distribution/etc. than our collective listening patterns/tastes? ([Chart rankings are based on sales (physical and digital), radio play, and online streaming in the United States.](https://www.billboard.com/pro/billboard-changes-streaming-weighting-hot-100-billboard-200/))

Check the same unique-songs-by-year chart as earlier, except now every year after the first year (1991) has fewer unique songs, since we've only kept the highest-charting week for each song in _the first year it appears_.

In [ ]:
_ = df.groupby(['year'])['artist_song'].count().plot.bar(title='Number of unique charting songs by year\n(w/o duplicate songs across years)')

For our final list of songs for which we'll scrape chords, take the top 100 songs from each year.

In [ ]:
df = df.groupby(['year']).head(100)

In [ ]:
_ = df.groupby(['year'])['artist_song'].count().plot.bar(title='Final song counts\n100 per year from 1991 to 2021')

In [ ]:
df

The dataframe `df` now holds all 3100 songs (the 100 highest-charting songs for the 31 years in 1991-2021) that we'll scrape chords for.

See the `billboard_reduce.py` script for the condensed version of the processing in this notebook.